![](https://seeklogo.com/images/N/nba-75-years-logo-7E38B318A3-seeklogo.com.png)

# **Initial EDA**
## CPSC322 Final Project
### Everett Johnson
### Luke Martin
### April 18, 2022
### NBA Team Success Predictor

In [1]:
import matplotlib.pyplot as plt
import os
from mysklearn.mypytable import MyPyTable
import mysklearn.myevaluation as myeval
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyNaiveBayesClassifier, MyDummyClassifier, MyDecisionTreeClassifier
from mysklearn.myensembleclassifiers import MyRandomForestClassifier

file_loc = os.path.join("input_data", "processed_data", "team_info.csv")
team_data = MyPyTable().load_from_file(file_loc)

win_percentage = team_data.get_column("Win Percentage")
#plt.figure()
#plt.hist(win_percentage, bins=5)
#plt.show()

file_loc = os.path.join("input_data", "processed_data", "team_stats.csv")
team_stats = MyPyTable().load_from_file(file_loc)

jppg = team_stats.get_column("JPPG")
#plt.figure()
#plt.hist(jppg, bins=10)
#plt.show()
#
trb = team_stats.get_column("TRB")
#plt.figure()
#plt.title("TRB")
#plt.hist(trb, bins=5)
#plt.show()
#
ast = team_stats.get_column("AST")
#plt.figure()
#plt.title("AST")
#plt.hist(ast, bins=5)
#plt.show()


In [2]:
y = team_stats.get_column("Success")
team_stats.drop_column("Success")
team_stats.drop_column("Team")
team_stats.drop_column("Season")
print(team_stats.column_names)
X = team_stats.data

['JPPG', 'TRB', 'AST', 'FG', '3FG']


In [3]:
from tabulate import tabulate
knn = MyKNeighborsClassifier(10)
print("KFold Acc:", myeval.kfold_accuracy(X, y, knn, n_splits=10, random_state=0))
print(tabulate(myeval.kfold_confusion_matrix(X, y, [1, 2, 3, 4, 5], knn, n_splits=10, random_state=0)))
dummy = MyDummyClassifier()
print("Dummy Acc:", myeval.kfold_accuracy(X, y, dummy, n_splits=10,random_state=0))

KFold Acc: 0.423841059602649
-  -  --  ---  ---  --  -----  ---------------
   1   2    3    4   5  Total  Recognition (%)
1  7  25   20    5   0  57     12
2  7  58  101   21   0  187    31
3  2  53  168   77   6  306    55
4  0  19  110  135  10  274    49
5  0   1   25   40  16  82     20
-  -  --  ---  ---  --  -----  ---------------
Dummy Acc: 0.33774834437086093


In [4]:
from tabulate import tabulate
forest = MyRandomForestClassifier(100, 10, 3)
print("Forest Acc:", myeval.kfold_accuracy(X, y, forest, n_splits=3, random_state=0))
print(tabulate(myeval.kfold_confusion_matrix(X, y, [1, 2, 3, 4, 5], forest, n_splits=3, random_state=0)))

Forest Acc: 0.39072847682119205
-  -  --  ---  ---  -  -----  ---------------
   1   2    3    4  5  Total  Recognition (%)
1  3  24   25    5  0  57     5
2  2  63   86   36  0  187    34
3  2  63  155   85  1  306    51
4  0  21  108  140  5  274    51
5  0   7   22   46  7  82     9
-  -  --  ---  ---  -  -----  ---------------


In [5]:
from tabulate import tabulate
tree = MyDecisionTreeClassifier()

header = ["att" + str(i) for i in range(len(X[0]))] + ["class"]
# storing data in MyPyTable object for access to functions there
train_data = [X[i] + [y[i]] for i in range(len(X))]
team_data_ii = MyPyTable(header, train_data)
attrib_domains = {}
for attribute in header:
    attrib_domains[attribute] = team_data_ii.get_atttribute_domain(attribute)
tree.attribute_domains = attrib_domains

print("Tree Acc:", myeval.kfold_accuracy(X, y, tree, n_splits=3, random_state=0))
print(tabulate(myeval.kfold_confusion_matrix(X, y, [1, 2, 3, 4, 5], tree, n_splits=3)))

Tree Acc: 0.38079470198675497
-  -  --  ---  --  -  -----  ---------------
   1   2    3   4  5  Total  Recognition (%)
1  0  21   36   0  0  57     0
2  0  40  139   8  0  187    21
3  3  28  231  44  0  306    75
4  1   1  209  62  1  274    23
5  0   0   51  30  1  82     1
-  -  --  ---  --  -  -----  ---------------


In [6]:
from tabulate import tabulate
nb = MyNaiveBayesClassifier()
print("NB Acc:", myeval.kfold_accuracy(X, y, nb, n_splits=3, random_state=0))
print(tabulate(myeval.kfold_confusion_matrix(X, y, [1, 2, 3, 4, 5], nb, n_splits=3, random_state=0)))

NB Acc: 0.4304635761589404
-  --  --  ---  ---  --  -----  ---------------
    1   2    3    4   5  Total  Recognition (%)
1  24  18   11    4   0  57     42
2  23  57   79   27   1  187    30
3  14  53  149   82   8  306    49
4   1  13   88  148  24  274    54
5   0   0   13   57  12  82     15
-  --  --  ---  ---  --  -----  ---------------
